# KNN (K-Nearest Neighbors)

KNN is a **supervised**, **instance-based** (lazy learner), and **non-parametric** algorithm used for classification and regression. It makes predictions by finding the $k$ closest data points (neighbors) in the training set to a new input and using a majority vote (classification) or an average (regression) of their values.



### Core Steps
The core steps for making a prediction on a new data point are:

1.  **Calculate Distance:** Measure the distance (e.g., Euclidean, Manhattan) from the new point to all points in the training set.
2.  **Find Neighbors:** Identify the $k$ points with the smallest distances.
3.  **Aggregate for Output:**
    * **For Classification:** Assign the most common class among the $k$ neighbors.
    * **For Regression:** Assign the average value of the $k$ neighbors.

---

## Critical Parameters and Hyperparameter Tuning
The performance of KNN hinges on several key decisions.

### 1. Choosing 'k' (Number of Neighbors)
This is the most critical parameter.
* **Small $k$ (e.g., 1):** Makes the model sensitive to noise (overfitting).
* **Large $k$:** Oversimplifies the model (underfitting).
* **Tip:** An **odd** $k$ value is preferred for classification to avoid ties. The optimal $k$ is data-dependent and is typically found using cross-validation or the Elbow Method.

### 2. Selecting a Distance Metric
The choice defines "closeness." Common metrics include:
* **Euclidean Distance:** The straight-line distance (default for continuous features).
* **Manhattan Distance:** Sum of absolute differences (useful for grid-like data).
* **Minkowski Distance:** A generalized formula; setting $p=2$ gives Euclidean, $p=1$ gives Manhattan.

### 3. Other Tuning Parameters
* **Weights:** Neighbors can be weighted uniformly or by the inverse of their distance (`weights='distance'`), giving closer points more influence.
* **Algorithm:** Methods like `ball_tree` or `kd_tree` can be faster than `brute` force for larger datasets.

> **Best Practice:** Use `GridSearchCV` or `RandomizedSearchCV` from scikit-learn to systematically test combinations of these parameters (e.g., `n_neighbors`, `weights`, `metric`) and find the best set via cross-validation.

---

## When to Use (and When to Avoid)

| Use KNN when... | Avoid KNN when... |
| :--- | :--- |
| You have a small to moderately sized dataset. | The dataset is very large (slow prediction). |
| Interpretability is important (explaining "nearest neighbors" is intuitive). | The dataset has many features (high-dimensional). |
| Data has a non-linear pattern and you need a simple baseline model. | Prediction speed is a critical requirement. |

---

## Real-World Applications
Due to its intuitive logic, KNN is widely used in:

* **Recommendation Systems:** Finding users with similar tastes to suggest products or content.
* **Pattern Recognition & Security:** Detecting fraudulent credit card transactions by identifying anomalous patterns.
* **Healthcare:** Classifying medical diagnoses, such as predicting the risk of a disease based on similar patient records.
* **Finance:** Credit scoring, stock market forecasting, and customer profiling.

# KNN: Classifier vs. Regressor

Just like Decision Trees, the KNN algorithm adapts based on whether your target variable is a **Category** (Label) or a **Number** (Value).

### 1. KNN Classifier
**Use this when:** Your target variable is **Categorical**.

* **The Logic:** "Majority Vote." The algorithm looks at the $k$ nearest neighbors and assigns the class that appears most frequently.
* **Tie-Breaking:** If there is a tie (e.g., 2 neighbors are Red, 2 are Blue), it often selects the class of the *closest* neighbor (if weights are used) or the first one found.
* **Output:** A Class Label (e.g., "Malignant", "Spam", "Blue").

**Example:**
* If $k=3$ and the neighbors are [Red, Red, Blue], the prediction is **Red**.

### 2. KNN Regressor
**Use this when:** Your target variable is **Continuous**.

* **The Logic:** "Average." The algorithm looks at the $k$ nearest neighbors and calculates the mean (average) of their values.
* **Output:** A Numerical Value (e.g., 150.5, $250k, 32°C).

**Example:**
* If $k=3$ and the neighbor values are [10, 20, 30], the prediction is **20** (since $(10+20+30)/3 = 20$).



---

### Comparison Table

| Feature | KNN **Classifier** | KNN **Regressor** |
| :--- | :--- | :--- |
| **Method** | Majority Voting | Averaging (Mean) |
| **Target** | Categories (Classes) | Continuous Numbers |
| **Evaluation** | Accuracy, F1-Score | MSE, MAE, $R^2$ |
| **Python Class** | `KNeighborsClassifier` | `KNeighborsRegressor` |

---

### Code Implementation

The setup is nearly identical, just a different class import.

```python
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

# -------------------------------------------------------
# SCENARIO 1: CLASSIFICATION (Predicting Fruit Type)
# -------------------------------------------------------
# X: [Weight, Color_Score]
# y: ['Apple', 'Orange', 'Apple']

# k=3, weights='distance' (closer neighbors have more say)
knn_clf = KNeighborsClassifier(n_neighbors=3, weights='distance')
# knn_clf.fit(X, y)


# -------------------------------------------------------
# SCENARIO 2: REGRESSION (Predicting House Price)
# -------------------------------------------------------
# X: [Sq_Ft, Num_Rooms]
# y: [250000, 400000, 320000]

# k=3, weights='uniform' (simple average)
knn_reg = KNeighborsRegressor(n_neighbors=3, weights='uniform')
# knn_reg.fit(X, y)

In [ ]:
# 1. Import Libraries & Load Data
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# df = pd.read_csv('your_data.csv')
# X = df.drop('target_column', axis=1)
# y = df['target_column']

# 2. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Feature Scaling (CRITICAL for KNN)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Hyperparameter Tuning with Grid Search
param_grid = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_scaled, y_train)

# 5. Evaluate Best Model
best_knn = grid_search.best_estimator_
y_pred = best_knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Params: {grid_search.best_params_}, Test Accuracy: {accuracy:.2f}")